# *PROCESO DE EXTRACCIÓN - TRANSFORMACIÓN - LIMPIEZA DE LOS DATOS DE GOOGLE MAPS Y YELP*

# IMPORTACIÓN DE LAS LIBRERIAS PARA EL PROCESO...

In [1]:
import os
import requests
import ast
import pandas as pd
import numpy as np
import json
import pyarrow.parquet as pq
import pickle

# CREACIÓN DE LOS DATAFRAMES CORRESPONDIENTES A CADA ARCHIVO...

In [2]:
# Archivo .parquet
archivo_parquet = 'datasets/yelp/user.parquet'

# Cargar datos desde el archivo .parquet
user_yelp = pd.read_parquet(archivo_parquet)

In [3]:
# Archivo .pkl
ruta_archivo = "datasets/yelp/business.pkl"

# Cargar el archivo .pkl en un DataFrame
business = pd.read_pickle(ruta_archivo)

In [4]:
# Archivo review
ruta_archivo = 'datasets/yelp/review.json'
review = pd.read_json(ruta_archivo, lines=True)
review = pd.json_normalize(review.to_dict(orient='records'))

In [5]:
# Archivo review
ruta_archivo = 'datasets/yelp/checkin.json'
checkin = pd.read_json(ruta_archivo, lines=True)
checkin = pd.json_normalize(checkin.to_dict(orient='records'))

In [6]:
# Archivo review
ruta_archivo = 'datasets/yelp/tip.json'
tip = pd.read_json(ruta_archivo, lines=True)
tip = pd.json_normalize(tip.to_dict(orient='records'))

# ANÁLISIS PRELIMINAR DE DATOS...

In [7]:
checkin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131930 entries, 0 to 131929
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   business_id  131930 non-null  object
 1   date         131930 non-null  object
dtypes: object(2)
memory usage: 2.0+ MB


In [8]:
checkin.dtypes

business_id    object
date           object
dtype: object

In [9]:
checkin.columns

Index(['business_id', 'date'], dtype='object')

In [10]:
checkin.shape

(131930, 2)

In [11]:
checkin.isnull().sum()

business_id    0
date           0
dtype: int64

In [12]:
checkin.astype(str).duplicated().sum()

0

In [13]:
review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6990280 entries, 0 to 6990279
Data columns (total 9 columns):
 #   Column       Dtype         
---  ------       -----         
 0   review_id    object        
 1   user_id      object        
 2   business_id  object        
 3   stars        int64         
 4   useful       int64         
 5   funny        int64         
 6   cool         int64         
 7   text         object        
 8   date         datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 480.0+ MB


In [14]:
review.dtypes

review_id              object
user_id                object
business_id            object
stars                   int64
useful                  int64
funny                   int64
cool                    int64
text                   object
date           datetime64[ns]
dtype: object

In [15]:
review.columns

Index(['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny',
       'cool', 'text', 'date'],
      dtype='object')

In [16]:
review.shape

(6990280, 9)

In [17]:
review.isnull().sum()

review_id      0
user_id        0
business_id    0
stars          0
useful         0
funny          0
cool           0
text           0
date           0
dtype: int64

In [18]:
review.astype(str).duplicated().sum()

0

In [19]:
tip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 908915 entries, 0 to 908914
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   user_id           908915 non-null  object        
 1   business_id       908915 non-null  object        
 2   text              908915 non-null  object        
 3   date              908915 non-null  datetime64[ns]
 4   compliment_count  908915 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 34.7+ MB


In [20]:
tip.dtypes

user_id                     object
business_id                 object
text                        object
date                datetime64[ns]
compliment_count             int64
dtype: object

In [21]:
tip.columns

Index(['user_id', 'business_id', 'text', 'date', 'compliment_count'], dtype='object')

In [22]:
tip.shape

(908915, 5)

In [23]:
tip.isnull().sum()

user_id             0
business_id         0
text                0
date                0
compliment_count    0
dtype: int64

In [24]:
tip.astype(str).duplicated().sum()

67

In [25]:
user_yelp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2105597 entries, 0 to 2105596
Data columns (total 22 columns):
 #   Column              Dtype  
---  ------              -----  
 0   user_id             object 
 1   name                object 
 2   review_count        int64  
 3   yelping_since       object 
 4   useful              int64  
 5   funny               int64  
 6   cool                int64  
 7   elite               object 
 8   friends             object 
 9   fans                int64  
 10  average_stars       float64
 11  compliment_hot      int64  
 12  compliment_more     int64  
 13  compliment_profile  int64  
 14  compliment_cute     int64  
 15  compliment_list     int64  
 16  compliment_note     int64  
 17  compliment_plain    int64  
 18  compliment_cool     int64  
 19  compliment_funny    int64  
 20  compliment_writer   int64  
 21  compliment_photos   int64  
dtypes: float64(1), int64(16), object(5)
memory usage: 353.4+ MB


In [26]:
user_yelp.dtypes

user_id                object
name                   object
review_count            int64
yelping_since          object
useful                  int64
funny                   int64
cool                    int64
elite                  object
friends                object
fans                    int64
average_stars         float64
compliment_hot          int64
compliment_more         int64
compliment_profile      int64
compliment_cute         int64
compliment_list         int64
compliment_note         int64
compliment_plain        int64
compliment_cool         int64
compliment_funny        int64
compliment_writer       int64
compliment_photos       int64
dtype: object

In [27]:
user_yelp.columns

Index(['user_id', 'name', 'review_count', 'yelping_since', 'useful', 'funny',
       'cool', 'elite', 'friends', 'fans', 'average_stars', 'compliment_hot',
       'compliment_more', 'compliment_profile', 'compliment_cute',
       'compliment_list', 'compliment_note', 'compliment_plain',
       'compliment_cool', 'compliment_funny', 'compliment_writer',
       'compliment_photos'],
      dtype='object')

In [28]:
user_yelp.shape

(2105597, 22)

In [29]:
user_yelp.isnull().sum()

user_id               0
name                  0
review_count          0
yelping_since         0
useful                0
funny                 0
cool                  0
elite                 0
friends               0
fans                  0
average_stars         0
compliment_hot        0
compliment_more       0
compliment_profile    0
compliment_cute       0
compliment_list       0
compliment_note       0
compliment_plain      0
compliment_cool       0
compliment_funny      0
compliment_writer     0
compliment_photos     0
dtype: int64

In [30]:
user_yelp.astype(str).duplicated().sum()

117700

In [31]:
business.info()

<class 'pandas.core.frame.DataFrame'>
Index: 150346 entries, 0 to 150345
Data columns (total 28 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   business_id   150346 non-null  object
 1   name          150346 non-null  object
 2   address       150346 non-null  object
 3   city          150346 non-null  object
 4   state         150343 non-null  object
 5   postal_code   150346 non-null  object
 6   latitude      150346 non-null  object
 7   longitude     150346 non-null  object
 8   stars         150346 non-null  object
 9   review_count  150346 non-null  object
 10  is_open       150346 non-null  object
 11  attributes    136602 non-null  object
 12  categories    150243 non-null  object
 13  hours         127123 non-null  object
 14  business_id   5 non-null       object
 15  name          5 non-null       object
 16  address       5 non-null       object
 17  city          5 non-null       object
 18  state         5 non-null     

In [32]:
business.dtypes

business_id     object
name            object
address         object
city            object
state           object
postal_code     object
latitude        object
longitude       object
stars           object
review_count    object
is_open         object
attributes      object
categories      object
hours           object
business_id     object
name            object
address         object
city            object
state           object
postal_code     object
latitude        object
longitude       object
stars           object
review_count    object
is_open         object
attributes      object
categories      object
hours           object
dtype: object

In [33]:
business.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours', 'business_id', 'name', 'address',
       'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars',
       'review_count', 'is_open', 'attributes', 'categories', 'hours'],
      dtype='object')

In [34]:
business.shape

(150346, 28)

In [35]:
business.isnull().sum()

business_id          0
name                 0
address              0
city                 0
state                3
postal_code          0
latitude             0
longitude            0
stars                0
review_count         0
is_open              0
attributes       13744
categories         103
hours            23223
business_id     150341
name            150341
address         150341
city            150341
state           150341
postal_code     150341
latitude        150341
longitude       150341
stars           150341
review_count    150341
is_open         150341
attributes      150341
categories      150341
hours           150341
dtype: int64

In [36]:
business.astype(str).duplicated().sum()

0

# PROCESO DE LIMPIEZA Y TRANSFORMACIÓN DE DATOS...

Información acerca de los datos del dataframe "review":
-**review_id**: "string" - id de reseña.  
-**user_id**: "string" - id unico de usuario y se relaciona con el archivo user.json.  
-**business_id**: "string" - id del negocio y se relaciona con el archivo business.pkl.
-**stars**: "entero" - puntaje de estrellas del UNO (01) al CINCO(05).
-**useful**: "entero" - numero de votos como reseña útil.
-**funny**: "entero" - numero de votos como reseña graciosa.
-**cool**: "entero" - numero de votos como reseña cool.  
-**text**: "string" - reseña en inglés.
-**date**: "string" - fecha con formato YYY - MM - DD.

In [38]:
# FUNCIÓN PARA TRABAJAR CON EL DATAFRAME review...

desanidados = []  # Lista vacía para almacenar los datos desanidados.

for _, row in review.iterrows():
    review_id = row['review_id']
    user_id = row['user_id']
    business_id = row['business_id']
    stars = row['stars']
    useful = row['useful']
    funny = row['funny']
    cool = row['cool']
    text = row['text']

    for datetime_str in str(row['date']).split(', '):
      datetime_parts = datetime_str.split(' ')
      date = datetime_parts[0]
      hour = datetime_parts[1] if len(datetime_parts) > 1 else None  # Usamos None si no hay hora disponible
      desanidados.append([review_id, user_id, business_id, stars, useful, funny, cool, text, date, hour])

review = pd.DataFrame(desanidados, columns=['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny', 'cool', 'text', 'date', 'hour'])


In [46]:
# RENOMBRAR COLUMNAS...

nuevos_nombres = {'review_id': 'ID_RESEÑA', 'user_id': 'ID_USUARIO', 'business_id': 'ID_NEGOCIO', 'stars': 'PUNTAJE', 'useful': 'VOTOS_RESEÑA_UTIL', 'funny': 'VOTOS_RESEÑA_GRACIOSO', 'text': 'RESEÑA', 'date': 'FECHA', 'hour': 'HORA'}
review = review.rename(columns=nuevos_nombres)

In [47]:
review.isnull().sum()

ID_RESEÑA                0
ID_USUARIO               0
ID_NEGOCIO               0
PUNTAJE                  0
VOTOS_RESEÑA_UTIL        0
VOTOS_RESEÑA_GRACIOSO    0
cool                     0
RESEÑA                   0
FECHA                    0
HORA                     0
dtype: int64

In [48]:
review.astype(str).duplicated().sum()

0

In [49]:
review.dtypes

ID_RESEÑA                object
ID_USUARIO               object
ID_NEGOCIO               object
PUNTAJE                   int64
VOTOS_RESEÑA_UTIL         int64
VOTOS_RESEÑA_GRACIOSO     int64
cool                      int64
RESEÑA                   object
FECHA                    object
HORA                     object
dtype: object

In [50]:
review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6990280 entries, 0 to 6990279
Data columns (total 10 columns):
 #   Column                 Dtype 
---  ------                 ----- 
 0   ID_RESEÑA              object
 1   ID_USUARIO             object
 2   ID_NEGOCIO             object
 3   PUNTAJE                int64 
 4   VOTOS_RESEÑA_UTIL      int64 
 5   VOTOS_RESEÑA_GRACIOSO  int64 
 6   cool                   int64 
 7   RESEÑA                 object
 8   FECHA                  object
 9   HORA                   object
dtypes: int64(4), object(6)
memory usage: 533.3+ MB


In [51]:
review.shape

(6990280, 10)

In [52]:
review.head(4)

,ID_RESEÑA,ID_USUARIO,ID_NEGOCIO,PUNTAJE,VOTOS_RESEÑA_UTIL,VOTOS_RESEÑA_GRACIOSO,cool,RESEÑA,FECHA,HORA
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07,22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03,15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05,20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04,00:01:03
